# Question Answering –  Fine-tuning – BERT-base


Ce notebook entraîne un modèle de Question Answering extractif
sur les données SQuAD préprocessées.


## Objectifs

- Charger les données préprocessées
- Charger un modèle QA pré-entraîné
- Fine-tuner avec Trainer
- Sauvegarder le meilleur modèle


In [1]:
from datasets import load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    DefaultDataCollator
)
import numpy as np


c:\Users\aissi\OneDrive - De Vinci\A5\UVSQ\S2\Fouille de données\qa-project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch

print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Nombre de GPUs: {torch.cuda.device_count()}")
else:
    print("Pas de GPU disponible - utilisation du CPU")

CUDA disponible: True
GPU: NVIDIA GeForce RTX 3060 Laptop GPU
Nombre de GPUs: 1


## Chargement des données préprocessées

Les données ont été prétraitées dans le notebook précédent et sont chargées directement depuis le disque.

**Note importante** : le fine-tuning est réalisé sur une version **réduite** du dataset SQuAD (`tokenized_squad_small`) afin de limiter le temps de calcul tout en conservant la structure réelle du problème de Question Answering. Cette approche est appropriée pour un projet académique avec ressources limitées.

In [4]:
tokenized_datasets = load_from_disk("outputs/tokenized_squad_small")

## Data collator

Permet de créer les batchs correctement.


In [5]:
data_collator = DefaultDataCollator()


## Tokenizer

Le tokenizer associé au modèle est chargé pour assurer la cohérence
du pipeline d’entraînement et d’inférence.


In [6]:
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


## Modèle

On charge un modèle compatible Question Answering.


In [7]:
model_checkpoint = "bert-base-uncased"
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)


Loading weights: 100%|██████████| 197/197 [00:00<00:00, 244.78it/s, Materializing param=bert.encoder.layer.11.output.dense.weight]              
BertForQuestionAnswering LOAD REPORT from: bert-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.dense.weight     | UNEXPECTED | 
bert.pooler.dense.weight                   | UNEXPECTED | 
bert.pooler.dense.bias                     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
qa_outputs.bias                            | MISSING    | 
qa_outputs.weight                          | MISSING    | 

Notes:
- UNEXPECTED	:can b

### Remarque sur les poids du modèle

Lors du chargement du modèle BERT-base, certains poids sont indiqués comme "MISSING" ou "UNEXPECTED" dans les logs. 

Cela est **normal et attendu** car :
- Le modèle pré-entraîné (`bert-base-uncased`) a été initialisé pour le language modeling sur un corpus général.
- La tête de Question Answering (`qa_outputs`) n'existe pas dans le modèle d'origine et est initialisée **aléatoirement** lors du chargement.
- Cette tête sera ensuite **entraînée** durant le fine-tuning sur les données SQuAD.

Aucune intervention n'est nécessaire : ce processus est standard dans le fine-tuning de modèles Transformers.

## Paramètres d’entraînement


In [ ]:
training_args = TrainingArguments(
    output_dir="outputs/checkpoints/bert",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="outputs/logs/bert",
    logging_steps=50,
    save_total_limit=1,
    load_best_model_at_end=True
)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


### Justification des hyperparamètres

Les hyperparamètres ont été choisis afin d'assurer un compromis optimal entre temps de calcul et performance :
- **Learning rate (3e-5)** : valeur standard recommandée pour le fine-tuning de modèles BERT
- **Batch size (16/8)** : taille de batch ambitieuse grâce à la disponibilité du GPU
- **1 epoch** : limite l'overfitting sur le dataset réduit tout en permettant l'apprentissage de la tête QA
- **Évaluation à chaque epoch** : permet le suivi de la performance en temps réel
- **save_total_limit=1** : conserve uniquement le meilleur modèle afin de limiter l'utilisation du disque

## Entraînement

On lance le fine-tuning avec Trainer.


In [ ]:
from transformers import set_seed

# Fixer la seed pour la reproductibilité
set_seed(42)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator
)

In [10]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.577216,2.861552


Writing model shards: 100%|██████████| 1/1 [00:02<00:00,  2.26s/it]
There were missing keys in the checkpoint model loaded: ['bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.output.LayerNorm.weight', 'bert.encoder.layer.0.output.LayerNorm.bias', 'bert.encoder.layer.1.attention.output.LayerNorm.weight', 'bert.encoder.layer.1.attention.output.LayerNorm.bias', 'bert.encoder.layer.1.output.LayerNorm.weight', 'bert.encoder.layer.1.output.LayerNorm.bias', 'bert.encoder.layer.2.attention.output.LayerNorm.weight', 'bert.encoder.layer.2.attention.output.LayerNorm.bias', 'bert.encoder.layer.2.output.LayerNorm.weight', 'bert.encoder.layer.2.output.LayerNorm.bias', 'bert.encoder.layer.3.attention.output.LayerNorm.weight', 'bert.encoder.layer.3.attention.output.LayerNorm.bias', 'bert.encoder.layer.3.output.LayerNorm.weight', 'bert.encoder.layer

TrainOutput(global_step=125, training_loss=3.923088623046875, metrics={'train_runtime': 442.9903, 'train_samples_per_second': 4.515, 'train_steps_per_second': 0.282, 'total_flos': 391945135104000.0, 'train_loss': 3.923088623046875, 'epoch': 1.0})

In [ ]:
import json

metrics_summary = {
    "train_loss": trainer.state.log_history[-1].get("train_loss", None),
    "eval_loss": trainer.state.log_history[-1].get("eval_loss", None),
    "learning_rate": training_args.learning_rate,
    "num_epochs": training_args.num_train_epochs,
    "batch_size": training_args.per_device_train_batch_size
}

with open("outputs/checkpoints/bert/training_metrics.json", "w") as f:
    json.dump(metrics_summary, f, indent=2)

print("\nMetriques sauvegardees dans outputs/checkpoints/bert/training_metrics.json")

## Sauvegarde

In [11]:
trainer.save_model("outputs/checkpoints/bert/final")
tokenizer.save_pretrained("outputs/checkpoints/bert/final")


Writing model shards: 100%|██████████| 1/1 [00:03<00:00,  3.69s/it]


('outputs/checkpoints/bert/final\\tokenizer_config.json',
 'outputs/checkpoints/bert/final\\tokenizer.json')

## Conclusion

Le modèle a été fine-tuné et sauvegardé.
La prochaine étape consistera à évaluer la qualité (EM/F1) et la latence,
puis à comparer 3 modèles.
